In [16]:
#!pip install python-dotenv
#!conda install -y -c conda-forge xgboost

You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [25]:
import sys
import os
import pandas as pd
from pathlib import Path
from datetime import datetime
import argparse
import psutil
import numpy as np
# import logging
import numpy as np
import tensorflow as tf
import time
import glob
from tensorflow.python.framework import ops
import math
import ftplib
from matplotlib import pyplot as plt
from collections import Counter
from dotenv import find_dotenv, load_dotenv
from scipy.stats.stats import pearsonr

import data_classes
import make_dataset as md
import build_data as bd
load_dotenv(find_dotenv())

pd.options.display.max_columns = 500
pd.options.display.max_rows = 500
pd.set_option('io.hdf.default_format','table')

In [26]:
RAW_DIR = os.path.join(Path(os.getcwd()).parents[0], 'XGBoost') 
PRO_DIR = os.path.join(Path(os.getcwd()).parents[0], 'XGBoost')
RANDOM_SEED = 123  # Set the seed to get reproducable results.
DT_FLOAT = np.float32
NP_FLOAT = np.dtype('float32')

train_dir = 'ImbDT_train' #'chuncks_random_c1mill_slices' #'chuncks_random_c1millx2_train'
valid_dir = 'ImbDS_valid'
test_dir = ''
train_period=[121,323] #[121,279] #[121, 143] 
valid_period=[324,329] #[280,285] #[144, 147] 
test_period=[330,342] #[286,304] #[148, 155]
batch_size=100000

print(RAW_DIR, PRO_DIR)
print(os.environ.get("FTP_PASS"))

/home/ec2-user/SageMaker/XGBoost /home/ec2-user/SageMaker/XGBoost
Mdjei3739


In [3]:
def download_file(rank, ftp_dir, local_dir, file_name):
    server = ftplib.FTP()
    server.connect(str(os.environ.get("FTP_HOST")), int(os.environ.get("FTP_PORT")))
    server.login(os.environ.get("FTP_USER"), "Mdjei3739#")     # os.environ.get("FTP_PASS")

    server.cwd(ftp_dir)               # change into ftp_dir directory
    print("FTP connection stablished by worker:  {}".format(rank))
    
    # filenames = server.nlst() # get filenames within the directory
        
    
    local_path = os.path.join(PRO_DIR, local_dir, file_name)             
    if not os.path.exists(local_path):            
        file = open(local_path, 'wb')
        server.retrbinary('RETR '+ file_name, file.write, 8*1024)            
        file.close()
        print("file downloaded:  {}".format(file_name))

    server.quit() # This is the “polite” way to close a connection
    print("FTP connection closed by worker:  {}".format(rank))

In [ ]:
#download_file(0, 'processed/random', train_dir, 'allTrans_12millRandNoInv_25mill-train_.h5')
#all_files = glob.glob(os.path.join(PRO_DIR, valid_dir, '*.h5'))
#print(len(all_files))

In [27]:
def get_files_dict(train_dir, valid_dir, test_dir):        
    ext = "*.h5"

    files_dict = {'train': glob.glob(os.path.join(PRO_DIR, train_dir, ext)), 
                  'valid': glob.glob(os.path.join(PRO_DIR, valid_dir, ext)), 
                  'test': glob.glob(os.path.join(PRO_DIR, test_dir, ext))}

    return files_dict

def architecture_settings(files_dict):
    architecture = {}    
    ok_inputs = True
    
    for key in files_dict.keys():
        total_records = 0
        for file in files_dict[key]:                                
            with pd.HDFStore(file) as dataset_file:
                if (ok_inputs): 
                    index_length = len(dataset_file.get_storer(key+'/features').attrs.data_columns)
                    architecture['n_input'] = dataset_file.get_storer(key+ '/features').ncols - index_length
                    architecture['n_classes'] = dataset_file.get_storer(key+'/labels').ncols - index_length
                    ok_inputs = False                
                total_records += dataset_file.get_storer(key + '/features').nrows
        architecture[key + '_num_examples'] = total_records                            
    
    architecture['total_num_examples'] = architecture['train_num_examples'] # 10000000
    return architecture

#To sum up the dataset per worker (assuming the same size of files per worker approximately):
files_dict = get_files_dict(train_dir, valid_dir, test_dir)
architecture = architecture_settings(files_dict)
print(architecture)

DATA = md.get_h5_data(PRO_DIR, architecture, None, valid_dir, None, train_period=train_period, valid_period=None, test_period=None)

{'n_input': 261, 'n_classes': 7, 'train_num_examples': 10076591, 'valid_num_examples': 343452, 'test_num_examples': 0, 'total_num_examples': 10076591}
/home/ec2-user/SageMaker/XGBoost/ImbDS_valid/allTrans_12millRandNoInv_25mill-valid_.h5 ...to load
/home/ec2-user/SageMaker/XGBoost/ImbDS_valid/allTrans_12millRandNoInv_25mill-valid_.h5  loaded in RAM
class_weights [  3659   7955   2592   5996 319262   3903     85]
class_weights [0.989 0.977 0.992 0.983 0.07  0.989 1.   ]


In [5]:
print(DATA.train._dict[0]['nrows'])
print(len(DATA.train.features_list), DATA.train.features_list)
print(len(DATA.train.labels_list), DATA.train.labels_list)
print(DATA.train._dict[0]['class_weights'])

In [28]:
if (DATA.validation._dict[0]):
    print(DATA.validation._dict[0]['nrows'])

343452


In [6]:
del DATA.train._dict[0]['dataset_features'][:1500000]
del DATA.train._dict[0]['dataset_labels'][:1500000]
del DATA.train._dict[0]['dataset_features'][1500000:]
del DATA.train._dict[0]['dataset_labels'][1500000:]
print(len(DATA.train._dict[0]['dataset_features']))

1500000


In [7]:
train_X = np.array(DATA.train._dict[0]['dataset_features']) #df_chunk.values
train_Y = np.argmax(np.array(DATA.train._dict[0]['dataset_labels']), axis=1) #y.values
print(train_X.shape)
print(train_Y.shape)
print('Training Dataset shape %s' % Counter(train_Y))

(1500000, 261)
(1500000,)
Training Dataset shape Counter({4: 1321115, 1: 50742, 3: 48568, 5: 40049, 0: 19358, 2: 19083, 6: 1085})


In [8]:
#Weighted Instances:
weighted_instances = np.take(DATA.train._dict[0]['class_weights'], train_Y)
print(DATA.train._dict[0]['class_weights'], train_Y[:10], weighted_instances[:10])

[0.987 0.972 0.99  0.976 0.094 0.981 0.999] [4 4 4 4 4 4 4 4 4 4] [0.094 0.094 0.094 0.094 0.094 0.094 0.094 0.094 0.094 0.094]


In [9]:
del DATA.train._dict[0]

In [29]:
test_X = np.array(DATA.validation._dict[0]['dataset_features'])
test_Y = np.argmax(np.array(DATA.validation._dict[0]['dataset_labels']), axis=1)
print(test_X.shape)
print(test_Y.shape)
print('Validation Dataset shape %s' % Counter(test_Y))

(343452, 261)
(343452,)
Validation Dataset shape Counter({4: 319262, 1: 7955, 3: 5996, 5: 3903, 0: 3659, 2: 2592, 6: 85})


In [31]:
#Weighted Instances:
weighted_instances = np.take(DATA.validation._dict[0]['class_weights'], test_Y)
print(DATA.validation._dict[0]['class_weights'], test_Y[:10], weighted_instances[:10])

[0.989 0.977 0.992 0.983 0.07  0.989 1.   ] [5 4 4 4 4 4 4 4 4 4] [0.989 0.07  0.07  0.07  0.07  0.07  0.07  0.07  0.07  0.07 ]


In [32]:
import sagemaker
import boto3
from sagemaker.predictor import csv_serializer    # Converts strings for HTTP POST requests on inference
from time import gmtime, strftime                 

 
region = boto3.Session().region_name    
smclient = boto3.Session().client('sagemaker')

2019-01-21 22:45:57,589 - botocore.hooks - DEBUG - Changing event name from creating-client-class.iot-data to creating-client-class.iot-data-plane
2019-01-21 22:45:57,594 - botocore.hooks - DEBUG - Changing event name from before-call.apigateway to before-call.api-gateway
2019-01-21 22:45:57,595 - botocore.hooks - DEBUG - Changing event name from request-created.machinelearning.Predict to request-created.machine-learning.Predict
2019-01-21 22:45:57,598 - botocore.hooks - DEBUG - Changing event name from before-parameter-build.autoscaling.CreateLaunchConfiguration to before-parameter-build.auto-scaling.CreateLaunchConfiguration
2019-01-21 22:45:57,598 - botocore.hooks - DEBUG - Changing event name from before-parameter-build.route53 to before-parameter-build.route-53
2019-01-21 22:45:57,599 - botocore.hooks - DEBUG - Changing event name from request-created.cloudsearchdomain.Search to request-created.cloudsearch-domain.Search
2019-01-21 22:45:57,601 - botocore.hooks - DEBUG - Changing e

In [33]:
print(region)
print(smclient)

us-east-2


In [34]:
from sagemaker import get_execution_role

role = sagemaker.get_execution_role()
print(role)

2019-01-21 22:46:04,515 - botocore.hooks - DEBUG - Changing event name from creating-client-class.iot-data to creating-client-class.iot-data-plane
2019-01-21 22:46:04,520 - botocore.hooks - DEBUG - Changing event name from before-call.apigateway to before-call.api-gateway
2019-01-21 22:46:04,521 - botocore.hooks - DEBUG - Changing event name from request-created.machinelearning.Predict to request-created.machine-learning.Predict
2019-01-21 22:46:04,524 - botocore.hooks - DEBUG - Changing event name from before-parameter-build.autoscaling.CreateLaunchConfiguration to before-parameter-build.auto-scaling.CreateLaunchConfiguration
2019-01-21 22:46:04,525 - botocore.hooks - DEBUG - Changing event name from before-parameter-build.route53 to before-parameter-build.route-53
2019-01-21 22:46:04,526 - botocore.hooks - DEBUG - Changing event name from request-created.cloudsearchdomain.Search to request-created.cloudsearch-domain.Search
2019-01-21 22:46:04,527 - botocore.hooks - DEBUG - Changing e

arn:aws:iam::099048829003:role/service-role/AmazonSageMaker-ExecutionRole-20181010T181868


In [35]:
# ARN of the bucket:
#arn:aws:s3:::sagemaker-eduardo-zea
bucket = 'sagemaker-eduardo-zea'
prefix = 'sagemaker/mortgage/imbalanced-weighted'
print(bucket, prefix)

sagemaker-eduardo-zea sagemaker/mortgage/imbalanced-weighted


In [14]:
train_Y = np.expand_dims(train_Y, axis=0)
weighted_instances = np.expand_dims(weighted_instances, axis=0)
print(train_Y, weighted_instances)

[[4 4 4 ... 3 3 4]] [[0.094 0.094 0.094 ... 0.976 0.976 0.094]]


In [15]:
train_X.shape

(1500000, 261)

In [16]:
xgb_train = np.concatenate((train_Y.T, weighted_instances.T, train_X), axis=1)

In [17]:
print(xgb_train.shape)
print(xgb_train[:10])

(1500000, 263)
[[ 4.          0.094      -0.33228114 ...  0.          0.
   0.        ]
 [ 4.          0.094      -0.33228114 ...  0.          0.
   0.        ]
 [ 4.          0.094      -0.33228114 ...  0.          0.
   0.        ]
 ...
 [ 4.          0.094      -0.33228114 ...  0.          0.
   0.        ]
 [ 4.          0.094      -0.33228114 ...  0.          0.
   0.        ]
 [ 4.          0.094      -0.33228114 ...  0.          0.
   0.        ]]


In [19]:
np.savetxt("/tmp/ImbDS3mill_train.csv", xgb_train, delimiter=",")

In [20]:
del train_X
del train_Y
del xgb_train

In [36]:
test_Y = np.expand_dims(test_Y, axis=0)
weighted_instances = np.expand_dims(weighted_instances, axis=0)
display(test_Y, weighted_instances)
xgb_test = np.concatenate((test_Y.T, weighted_instances.T,  test_X), axis=1)

array([[5, 4, 4, ..., 4, 0, 4]])

array([[0.989, 0.07 , 0.07 , ..., 0.07 , 0.989, 0.07 ]])

In [37]:
print(xgb_test.shape)

(343452, 263)


In [ ]:
np.savetxt("/tmp/ImbDS3mill_valid.csv", xgb_test, delimiter=",") # fmt='%.18e' # '%10.5f'
# xgb_test.tofile("bal100th_valid.csv", format='%10.5f')

In [ ]:
#del test_X
# del test_Y
#del xgb_test
# del DATA.validation._dict[0]

In [ ]:
#saving a test set that is the same as validation: Change the next time!
np.savetxt("/tmp/bal100th_test.csv", test_X, delimiter=",")

In [24]:
#!rm /tmp/ImbDS3mill_train.csv
#!rm /tmp/ImbDS3mill_valid.csv
!dir -l /tmp/

total 9674208
drwxr-xr-x 2 role-agent role-agent       4096 Jan 21 14:15 hsperfdata_role-agent
-rw-rw-r-- 1 ec2-user   ec2-user   9906373885 Jan 21 21:01 ImbDS3mill_train.csv
drwxr-xr-x 3 role-agent role-agent       4096 Jan 21 14:15 jetty-0.0.0.0-9081-role-proxy-agent.war-_-any-3805171590929210667.dir


In [18]:
ret_train = np.loadtxt("/tmp/ImbDS3mill_train.csv", delimiter=",")

In [22]:
print(ret_train.shape, ret_train[:10])

(1500000, 263) [[ 4.          0.094      -0.33228114 ...  0.          0.
   0.        ]
 [ 4.          0.094      -0.33228114 ...  0.          0.
   0.        ]
 [ 4.          0.094      -0.33228114 ...  0.          0.
   0.        ]
 ...
 [ 4.          0.094      -0.33228114 ...  0.          0.
   0.        ]
 [ 4.          0.094      -0.33228114 ...  0.          0.
   0.        ]
 [ 4.          0.094      -0.33228114 ...  0.          0.
   0.        ]]


In [20]:
ret_valid = np.loadtxt("/tmp/ImbDS3mill_valid.csv", delimiter=",")

In [21]:
print(ret_valid.shape, ret_valid[:10])

(343452, 262) [[ 5.          3.45559764  0.         ...  0.          0.
   0.        ]
 [ 4.         -0.33228114  0.         ...  0.          0.
   0.        ]
 [ 4.         -0.33228114  0.         ...  0.          0.
   0.        ]
 ...
 [ 4.         -0.33228114  0.         ...  0.          0.
   0.        ]
 [ 4.         -0.33228114  0.         ...  0.          0.
   0.        ]
 [ 4.         -0.33228114  0.         ...  0.          0.
   0.        ]]


In [42]:
#frozen:
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train_1.csv')).upload_file('/tmp/ImbDS3mill_train.csv')
#s3_path = 's3://'+ bucket + '/' + prefix + '/train/train.csv'
#print(s3_path)
#!aws s3 cp /tmp/bal100th_train.csv s3_path #it does not work!
#copy all files in my-data-dir into the "data" directory located in my-s3-bucket 
#aws s3 cp my-data-dir/ s3://my-s3-bucket/data/ --recursive

2019-01-21 18:03:56,731 - botocore.hooks - DEBUG - Changing event name from creating-client-class.iot-data to creating-client-class.iot-data-plane
2019-01-21 18:03:56,736 - botocore.hooks - DEBUG - Changing event name from before-call.apigateway to before-call.api-gateway
2019-01-21 18:03:56,737 - botocore.hooks - DEBUG - Changing event name from request-created.machinelearning.Predict to request-created.machine-learning.Predict
2019-01-21 18:03:56,740 - botocore.hooks - DEBUG - Changing event name from before-parameter-build.autoscaling.CreateLaunchConfiguration to before-parameter-build.auto-scaling.CreateLaunchConfiguration
2019-01-21 18:03:56,740 - botocore.hooks - DEBUG - Changing event name from before-parameter-build.route53 to before-parameter-build.route-53
2019-01-21 18:03:56,741 - botocore.hooks - DEBUG - Changing event name from request-created.cloudsearchdomain.Search to request-created.cloudsearch-domain.Search
2019-01-21 18:03:56,743 - botocore.hooks - DEBUG - Changing e

FileNotFoundError: [Errno 2] No such file or directory: '/tmp/bal100th_train.csv'

In [16]:
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('/tmp/ImbDS3mill_valid.csv')

2019-01-21 21:35:12,690 - botocore.hooks - DEBUG - Changing event name from creating-client-class.iot-data to creating-client-class.iot-data-plane
2019-01-21 21:35:12,695 - botocore.hooks - DEBUG - Changing event name from before-call.apigateway to before-call.api-gateway
2019-01-21 21:35:12,696 - botocore.hooks - DEBUG - Changing event name from request-created.machinelearning.Predict to request-created.machine-learning.Predict
2019-01-21 21:35:12,699 - botocore.hooks - DEBUG - Changing event name from before-parameter-build.autoscaling.CreateLaunchConfiguration to before-parameter-build.auto-scaling.CreateLaunchConfiguration
2019-01-21 21:35:12,699 - botocore.hooks - DEBUG - Changing event name from before-parameter-build.route53 to before-parameter-build.route-53
2019-01-21 21:35:12,700 - botocore.hooks - DEBUG - Changing event name from request-created.cloudsearchdomain.Search to request-created.cloudsearch-domain.Search
2019-01-21 21:35:12,702 - botocore.hooks - DEBUG - Changing e

In [ ]:
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'prediction/test.csv')).upload_file('/tmp/bal100th_test.csv')

In [ ]:
#"TrainingJobEarlyStoppingType" : "Auto"
tuning_job_config = {
    "ParameterRanges": {
      "CategoricalParameterRanges": [],
      "ContinuousParameterRanges": [
        {
          "MaxValue": "1",
          "MinValue": "0.3",
          "Name": "eta"
        },
        {
          "MaxValue": "2",
          "MinValue": "1",
          "Name": "alpha"
        },
        {
          "MaxValue": "15",
          "MinValue": "5",
          "Name": "min_child_weight"
        }
      ],
      "IntegerParameterRanges": [
        {
          "MaxValue": "30",
          "MinValue": "15",
          "Name": "max_depth"
        },
        {
          "MaxValue": "10",
          "MinValue": "3",
          "Name": "max_delta_step"
        }
          
      ]
    },
    "ResourceLimits": {
      "MaxNumberOfTrainingJobs": 10,
      "MaxParallelTrainingJobs": 2
    },
    "Strategy": "Bayesian",
    "HyperParameterTuningJobObjective": {
      "MetricName": "validation:merror",
      "Type": "Minimize"
    },   
    "TrainingJobEarlyStoppingType" : "Auto"
  } 

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri
training_image = get_image_uri(boto3.Session().region_name, 'xgboost')

s3_input_train = 's3://{}/{}/train'.format(bucket, prefix)
s3_input_validation ='s3://{}/{}/validation/'.format(bucket, prefix)
     
training_job_definition = {
    "AlgorithmSpecification": {
      "TrainingImage": training_image,
      "TrainingInputMode": "File"
    },
    "InputDataConfig": [
      {
        "ChannelName": "train",
        "CompressionType": "None",
        "ContentType": "csv",
        "DataSource": {
          "S3DataSource": {
            "S3DataDistributionType": "FullyReplicated", #"ShardedByS3Key",
            "S3DataType": "S3Prefix",
            "S3Uri": s3_input_train
          }
        }
      },
      {
        "ChannelName": "validation",
        "CompressionType": "None",
        "ContentType": "csv",
        "DataSource": {
          "S3DataSource": {
            "S3DataDistributionType": "FullyReplicated", #"ShardedByS3Key",
            "S3DataType": "S3Prefix",
            "S3Uri": s3_input_validation
          } 
        }
      }
    ],
    "OutputDataConfig": {
      "S3OutputPath": "s3://{}/{}/output".format(bucket,prefix)
    },
    "ResourceConfig": {
      "InstanceCount": 2,
      "InstanceType": "ml.m5.2xlarge", #"ml.c4.2xlarge",
      "VolumeSizeInGB": 15 # the platform download the entire file for each 
    },
    "RoleArn": role,
    "StaticHyperParameters": {
      "eval_metric": "merror",  # "auc",
      "num_round": "15",
      "objective": "multi:softmax", #"rate_drop": "0.3",
      "tweedie_variance_power": "1.4",
      "num_class": "7"
    },
    "StoppingCondition": {
      "MaxRuntimeInSeconds": 430000
    }
}
tuning_job_name = "XGBmortgages013"

In [ ]:
smclient.create_hyper_parameter_tuning_job(HyperParameterTuningJobName = tuning_job_name,
                                           HyperParameterTuningJobConfig = tuning_job_config,
                                           TrainingJobDefinition = training_job_definition)

In [ ]:
tuner_parent_metrics = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)
if not tuner_parent_metrics.dataframe().empty:
    df_parent = tuner_parent_metrics.dataframe().sort_values(['FinalObjectiveValue'], ascending=True)    
df_parent

In [ ]:
import bokeh
import bokeh.io
bokeh.io.output_notebook()
from bokeh.plotting import figure, show
from bokeh.models import HoverTool

import pandas as pd

df_parent_objective_value = df_parent[df_parent['FinalObjectiveValue'] > -float('inf')]

p = figure(plot_width=900, plot_height=400, x_axis_type='datetime',x_axis_label='datetime', y_axis_label='validation:merror')
p.circle(source=df_parent_objective_value, x='TrainingStartTime', y='FinalObjectiveValue', color='black')

show(p)

In [ ]:
#from sagemaker.tuner import WarmStartConfig, WarmStartTypes
#parent_tuning_job_name = tuning_job_name
#warm_start_config = WarmStartConfig(WarmStartTypes.IDENTICAL_DATA_AND_ALGORITHM, parents={parent_tuning_job_name})
#print(warm_start_config)

#Using boto3 API:
warm_start_config = {
          "ParentHyperParameterTuningJobs" : [ 
          {"HyperParameterTuningJobName" :'XGBmortgages012'}, 
          {"HyperParameterTuningJobName" :'XGBmortgages013'}
          ],
          "WarmStartType" : "IdenticalDataAndAlgorithm"    
}

In [ ]:
smclient.create_hyper_parameter_tuning_job(HyperParameterTuningJobName = 'XGBmortgages014',
   HyperParameterTuningJobConfig = tuning_job_config, # See notebook for tuning configuration
   TrainingJobDefinition = training_job_definition, # See notebook for job definition
   WarmStartConfig = warm_start_config)